In [103]:
from lxml import etree
import re
import os
import sqlite3

In [104]:
def get_entries_from_link(fr=0, \
                          maximum=1000, \
                          baselink=None):
    if baselink is None:
        baselink = 'http://data.rechtspraak.nl/uitspraken/zoeken?return=DOC&creator=http://standaarden.overheid.nl/owms/terms/Hoge_Raad_der_Nederlanden'
    link = baselink+'&max='+str(maximum)+'&from='+str(fr)
    xml_element = etree.ElementTree().parse(link)
    entries = list(xml_element.iter('{*}entry'))
    return entries

## Make a database

In [105]:
conn = sqlite3.connect('rechtspraak.db')

In [106]:
c = conn.cursor()

In [108]:
c.execute(''' DROP TABLE IF EXISTS uitspraken_meta''')
c.execute(''' CREATE TABLE uitspraken_meta
            (id text PRIMARY KEY, 
            title text,
            summary text,
            updated text
            )
        ''')

Now populate the table

In [109]:
def get_first_content(el, tag):
    return list(el.iter('{*}'+tag))[0].text

def insert_into_uitspraken_meta(entry, curs, table='uitspraken_meta'):
    id0 = get_first_content(entry, 'id')
    title = get_first_content(entry, 'title')
    summary = get_first_content(entry, 'summary')
    updated = get_first_content(entry, 'updated')
    query = ''' INSERT OR REPLACE INTO uitspraken_meta
        VALUES (?, ?, ?, ?)
    '''
    curs.execute(query, (id0, title, summary, updated))

In [110]:
size = 1000
end = 30000
for start in range(0, end, size):
    entries = get_entries_from_link(start, size)
    print(start, len(entries))
    for entry in entries:
        insert_into_uitspraken_meta(entry, c)

(0, 1000)
(1000, 1000)
(2000, 1000)
(3000, 1000)
(4000, 1000)
(5000, 1000)
(6000, 1000)
(7000, 1000)
(8000, 1000)
(9000, 1000)
(10000, 1000)
(11000, 1000)
(12000, 1000)
(13000, 1000)
(14000, 1000)
(15000, 1000)
(16000, 1000)
(17000, 1000)
(18000, 1000)
(19000, 1000)
(20000, 1000)
(21000, 1000)
(22000, 1000)
(23000, 1000)
(24000, 1000)
(25000, 1000)
(26000, 1000)
(27000, 1000)
(28000, 1000)
(29000, 39)


In [111]:
conn.commit()

In [114]:
conn.close()